Imports.

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

Plot settings.

In [ ]:
DPI = 300
FIGSIZE = (12,4)
FONTSIZE = 16

# Billinge group colors
COLORS = dict(bg_blue="#0B3C5D", 
              bg_red="#B82601", 
              bg_green="#1c6b0a",
              bg_lightblue="#328CC1", 
              bg_darkblue="#062F4F", 
              bg_yellow="#D9B310",
              bg_darkred="#984B43", 
              bg_bordeaux="#76323F", 
              bg_olivegreen="#626E60",
              bg_yellowgrey="#AB987A", 
              bg_brownorange="#C09F80",
              )
COLOR = COLORS["bg_blue"]

Function to convert .asc to .csv and plotting data.

In [ ]:
def asc_to_csv_converter_plotter(files, csvpath, plotpaths):
    for e in files:
        print(f"\t{e.name}")
        filename = e.stem
        with open(e) as f:
            lines = f.readlines()
        for i in range(len(lines)):
            if "*START" in lines[i]:
                start_tt = float(lines[i].split()[-1])
            elif "*STOP" in lines[i]:
                stop_tt = float(lines[i].split()[-1])
            elif "*STEP" in lines[i]:
                step_tt = float(lines[i].split()[-1])
            elif "*COUNT" in lines[i]:
                start_int = i+1
            elif "*END" in lines[i]:
                end_int = i
        tt = np.arange(start_tt, stop_tt+step_tt, step_tt)
        int_exp = []
        for i in range(start_int, end_int):
            for j in range(len(lines[i].split(","))):
                int_exp.append(float(lines[i].split(",")[j]))
        int_exp = np.array(int_exp)
        tt_int_exp = np.column_stack((tt, int_exp))
        print(f"\t\t{csvpath.name}")
        np.savetxt(csvpath / f"{filename}.csv", 
                   tt_int_exp, 
                   delimiter=",", 
                   fmt="%.3f",
                   )
        fig, ax = plt.subplots(figsize=FIGSIZE)
        ax.plot(tt, int_exp, c=COLOR)
        ax.set_xlim(np.amin(tt), np.amax(tt))
        ax.set_xlabel(r"$2\theta$ $[\degree]$", fontsize=FONTSIZE)
        ax.set_ylabel(r"$I$ $[\mathrm{counts}]$", fontsize=FONTSIZE)
        ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        for p in plotpaths:
            print(f"\t\t{p.name}")
            plt.savefig(p / f"{filename}.{p.name}", bbox_inches="tight")
        plt.close()

    return None

Creating ouput paths.

In [ ]:
folders = ["csv", "png", "pdf", "svg"]
paths = [Path.cwd() / folder for folder in folders]
for p in paths:
    if not p.exists():
        p.mkdir()
csvpath = Path.cwd() / "csv"
plotpaths = [p for p in paths if not p.name == "csv"]
plotfolders = [p.name for p in paths if not p.name == "csv"]

Collecting data .asc files.

In [ ]:
if not (Path.cwd() / 'asc').exists():
    (Path.cwd() / 'asc').mkdir()
    print(f"{80*'-'}\nA folder called 'asc' has been made. Please place your files there and rerun the code.\
            \n{80*'-'}")
    sys.exit()
files = list((Path.cwd() / 'asc').glob("*.asc"))
if len(files) == 0:
    print(f"{80*'-'}\nNo .asc files found in the 'asc' folder. Please place your files there and rerun the code.\
            \n{80*'-'}")
    sys.exit()

Converting asc files to csv files and plotting.

In [ ]:
print(f"{80*'-'}\nConverting asc files to csv files and plotting...")
asc_to_csv_converter_plotter(files, csvpath, plotpaths)
print(f"{80*'-'}\n.asc files have been converted to .csv files and saved to "
        f"the 'csv' folder.\nPlots have been saved to the {plotfolders} "
        f"folders.\n{80*'-'}")